In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import pandas as pd
import cv2
import os
import numpy as np
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [0]:
torch.manual_seed(1618)

## Intialize parameters

In [0]:
root_dir = '/content/gdrive/My Drive/Summer_19/eece5626/'

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## CLAHE

In [0]:
class CLAHE(object):
    def __init__(self, clip_limit):
        assert isinstance(clip_limit, float) or isinstance(clip_limit, int)
        self.clip_limit = clip_limit
    
    def __call__(self, sample):
        image, diagnosis = sample['image'], sample['diagnosis']
        
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit)
        img = clahe.apply(image)
        return {"image": img, "diagnosis": diagnosis}

## Data Loader

In [0]:
class APTOSDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.aptos_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.aptos_frame)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.aptos_frame.iloc[idx, 0])
        image = cv2.imread(img_name + ".png", 0)
        
        image = cv2.resize(image, (224, 224))
        
        diagnosis = self.aptos_frame.iloc[idx, 1]
        diagnosis = np.array([diagnosis])
        
        sample = {'image': image, 'diagnosis': diagnosis, 'id': img_name}
        
        clahe = CLAHE(clip_limit=3)
        sample = clahe(sample)
        
        if self.transform:
            sample["image"] = self.transform(sample["image"])
            
        return sample

## AlexNet Model

In [0]:
class MyAlexNet(object):
  def __init__(self, lr, wd):
    # model
    self.model = models.alexnet(pretrained=True)
    for param in self.model.parameters():
      param.requires_grad = False
      self.model.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    self.model.classifier._modules['6'] = nn.Linear(4096, 2)
    self.model.to(device)
    
    # hyper params
    self.lr = lr
    self.wd = wd
    
    # optimizer
    self.optimizer = optim.Adam(self.model.parameters(), lr=lr,weight_decay=wd, amsgrad=True)
    self.criterion = nn.CrossEntropyLoss()  

    # model stats
    self.incorrect_classifications = []
    
  def train(self, data, n_epochs=1):
    self.model.train()
    for i in range(n_epochs):
      for j, sample_batched in enumerate(data, 1):
        # inputs and labels
        inputs = sample_batched['image'].to(device)
        labels = sample_batched['diagnosis'].squeeze(1).to(device)
        
        # optimizer
        self.optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = self.model(inputs)
        loss = self.criterion(F.log_softmax(outputs, dim=1), labels)
        loss.backward()
        self.optimizer.step()
      
      
      print("Epoch ", (i+1), "/", n_epochs)
        
  def predict(self, data):
    self.model.eval()
    predictions = []
    labels = []
    img_names = []
    for i, sample_batched in enumerate(data, 1):
      inp = sample_batched["image"].to(device)
      lab = sample_batched["diagnosis"]
      img_name = sampled_batched["id"]
      op = self.model(inp)
      pred = op.argmax(dim=1)
      predictions.extend(pred)
      labels.extend(lab)
      img_names.extend(img_name)
      
      
    results = {}
    results["true_y"] = [el.item() for el in labels]
    results["predicted_y"] = [el.item() for el in predictions]
    results["image_names"] = [el.item() for el in img_names]
    return results
  
  def export(self, filename="myalexnet.pth"):
    torch.save(self.model, filename)

## Training the model

In [0]:
train_path = root_dir + "training_01.csv"

In [0]:
images_path = root_dir + "train_preprocessed/"

In [0]:
aptos_train = APTOSDataset(csv_file=train_path,
                             root_dir=images_path,
                             transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

In [0]:
trainLoader = torch.utils.data.DataLoader(aptos_train, batch_size=64)

In [0]:
lr= 0.0003
w = 6.7752e-06
my_alexnet = MyAlexNet(lr, w)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth
100%|██████████| 244418560/244418560 [00:02<00:00, 105328854.81it/s]


In [0]:
my_alexnet.train(trainLoader, n_epochs=5)

Epoch  1 / 5
Epoch  2 / 5
Epoch  3 / 5
Epoch  4 / 5
Epoch  5 / 5


In [0]:
results_train = my_alexnet.predict(trainLoader)

### Training results

**Confusion Matrix**

In [0]:
cm_train = confusion_matrix(results_train["true_y"], results_train["predicted_y"])

In [0]:
cm_train

**F1-score**

In [0]:
f1_train = f1_score(results_train["true_y"], results_train["predicted_y"])

In [0]:
f1_train

## Validating the model

In [0]:
validation_path = root_dir + "validation_01.csv"

In [0]:
images_path = root_dir + "train_preprocessed/"

In [0]:
aptos_validation = APTOSDataset(csv_file=validation_path,
                             root_dir=images_path,
                             transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

In [0]:
validationLoader = torch.utils.data.DataLoader(aptos_validation)

In [0]:
results_validation = my_alexnet.predict(validationLoader)

### Validation results

**Confusion Matrix**

In [0]:
cm_validation = confusion_matrix(results_validation["true_y"], results_validation["predicted_y"])

In [0]:
cm_validation

**F1-score**

In [0]:
f1_validation = f1_score(results_validation["true_y"], results_validation["predicted_y"])

In [0]:
f1_validation

In [0]:
# torch.save(my_alexnet.model, root_dir + "alexnet_model.pth")